#  Preparation

### Modules

In [1]:
using JuMP, Gurobi

### Solver

In [2]:
solver = GurobiSolver(Presolve = 1)
;

### Data

In [3]:
c = [2, 1, 2, 4]    # Cost of delays for each plane
t = [1, 2, 2, 3]    # Ideal arrival time
l = [2, 0.5, 1, 2]  # Time spent on the runway
set_up = [  [0 5 2 3]
            [1 0 2 3]
            [2 0 3 4]
            [2 0 4 4]  ] # Set-up times: entry (p,q) is the minimum time required between the finish of plane p and the arrival of plane q
d_max = [20, 20, 20, 20] # Maximum allowable delay for each plane
;

### Pre-checks

In [4]:
P = length(c) # Number of planes
assert(all(l.>=0))
assert(length(t)==P)
assert(length(l)==P)
assert(length(d_max)==P)
assert(size(set_up)==(P,P))

### Big M calculation

In [5]:
BigM = Array(Float64,P,P)
for p = 1:P
    for q = 1:P
        BigM[p,q] = 2*d_max[p] + 2*d_max[q] + l[p]+l[q] + 2*abs(t[p]-t[q]) 
    end
end

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] Array(::Type{Float64}, ::Int64, ::Int64) at ./deprecated.jl:57
 [3] include_string(::String, ::String) at ./loading.jl:522
 [4] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /home/nathan/.julia/v0.6/IJulia/src/execute_request.jl:158
 [5] (::Compat.#inner#17{Array{Any,1},IJulia.#execute_request,Tuple{ZMQ.Socket,IJulia.Msg}})() at /home/nathan/.julia/v0.6/Compat/src/Compat.jl:385
 [6] eventloop(::ZMQ.Socket) at /home/nathan/.julia/v0.6/IJulia/src/eventloop.jl:8
 [7] (::IJulia.##14#17)() at ./task.jl:335
while loading In[5], in expression starting on line 1


We would like $M_{p,q} \geq W_{p,q}$.

$$\begin{align*}
    W_{p,q} &= \max\{L_{p,q},L_{q,p}\}\\
            &\leq |L_{p,q}| + |L_{q,p}|\\
            &= |e_p - a_q| + |e_q - a_p|\\
            &= |l_p + a_p - a_q| + |l_q + a_q - a_p|\\
            &\leq | l_p| + |a_p - a_q| + |l_q| + |a_q - a_p|&\text{By the triangle inequality}\\
            &= l_p +  l_q + 2| a_p - a_q|&\text{since }l\geq 0\\
            &= l_p +  l_q + 2| t_p+d_p-t_q-d_q|\\
            &\leq l_p +  l_q + 2| t_p-t_q| + 2|d_p-d_q|&\text{By the triangle inequality}\\
            &\leq 2|d_p|+2|d_q|+l_p +  l_q + 2| t_p-t_q|&\text{By the triangle inequality}\\
            &\leq 2d_{\max_p}+2d_{\max_q}+l_p +  l_q + 2| t_p-t_q|\\
            &=:M_{p,q}
\end{align*}$$

# Model

### Main model object

In [6]:
m = Model(solver = solver)
;

### Variables 

In [7]:
@variable(m, a[1:P]) # Scheduled arrival times
@variable(m, e[1:P]) # Scheduled ending times
@variable(m, d[1:P] >= 0) # Scheduled delays
@variable(m, L[1:P,1:P]) # Time between the first plane, p, finishing and the second, q, arriving
@variable(m, W[1:P,1:P] >= 0) # The smallest window of time that contains all intervals of time in which either plane p or plane q is landing.
@variable(m, G[1:P,1:P] >= 0) # The gap between plane p ad q's landings
@variable(m, A[1:P,1:P], Bin) # Binary matrix; entry (p,q) is equal to 1 if plane p arrives before plane q, and 0 otherwise.
;

### Objective

In [8]:
@objective(m, Min, dot(c,d)) # Minimise the total cost of delays
;

### Essential Constraints

In [9]:
@constraint(m, a .== t + d) # The arrival time is the ideal arrival time, plus delay
@constraint(m, e .== a + l) # The ending time is the arrival time plus time spent on runway
@constraint(m, d .<= d_max) # Maximum delay
for p = 1:P
    for q = 1:P
        @constraint(m, L[p,q] == e[p] - a[q]) # By the definition of L
        
        @constraint(m, W[p,q] >= L[p,q]) # W[p,q] == maximum(L[p,q], L[q,p]), constraint I
        @constraint(m, W[p,q] >= L[q,p]) # W[p,q] == maximum(L[p,q], L[q,p]), constraint II
        @constraint(m, W[p,q] <= L[p,q] + BigM[p,q]*A[p,q]) # W[p,q] == maximum(L[p,q], L[q,p]), constraint III
        @constraint(m, W[p,q] <= L[q,p] + BigM[p,q]*(1-A[p,q])) # W[p,q] == maximum(L[p,q], L[q,p]), constraint IV

        if p != q
            @constraint(m, G[p,q]+l[p]+l[q] == W[p,q]) # The window W is the time the two planes spend each on the runway, plus the gap between these planes.
            @constraint(m, G[p,q] >= set_up[p,q].*A[p,q]) # Enforce set-up times
            @constraint(m, A[p,q] + A[q,p] == 1) # Plane p comes before plane q, or q before p, but not both.
        end
    end
end

### Additional Constraints

Now for some constraints that should help make the problem easier to solve, but aren't necessary in enforcing the logic of the problem. The first is to get an upper bound on each delay. In the very worst case scenario, plane $p$ ideally arrives first, but gets scheduled to arrive last, after all the other planes have arrived one after the other, incurring times of
$$\sum_{i\neq p} l_i+(P-1)\max_{i,j}\mathrm{(Setup)}_{i,j}$$
in total. Therefore our constraint is

In [10]:
for p = 1:P
    @constraint(m, d[p] <= sum(l)-l[p] + (P-1)*maximum(set_up))
end

Given a sequence of planes $p,q,r,s...$, plane $p$ comes before all of them and so will have $P-1$ ones in its row of the $A$ matrix.
Plane $q$ has one fewer, $P-2$, and plane $r$ has $P-3$. This continues up until the final plane numbered $P$, which has all zeros.
The total number of ones in the $A$ matrix is therefore
$$\begin{align*}
    \sum_{p=1}^{P} (P-p) &= \sum_{p=1}^{P} P - \sum_{p=1}^{P} p\\
					   &= P^2 - P(P+1)/2\\
					   &= P(P-1)/2
\end{align*}$$

In [11]:
@constraint(m, sum(A)==P*(P-1)/2)
;

"Coming before" is a transitive property. What this means is that if p comes before q, and q comes before r, then p comes before r. This fact can put some more constraints on the A matrix.

In [12]:
for p = 1:P
    for q = 1:P
        for r = 1:P
            @constraint(m, A[p,q] + A[q,r] <= 1 + A[p,r])
        end
    end
end
;

The idea behind this formulation is that if the planes land one after the other you will have a situation like this:

<img src="files/no_overlap.png"> 

whereas with overlap you have this:

<img src="files/yes_overlap.png"> 



 In the first case, $W_{p,q} \geq l_p + l_q$, while in the second, that inequality is violated. Since $G \geq 0$, this forces no overlap.
Notice also how $W_{p,q} = \max(L_{p,q}, L_{q,p})$. At least one of the two arguments must be positive, so $W$ must be too.

# Solve

### Run solver

In [18]:
tic();
status = solve(m);
toc();

Optimize a model with 197 rows, 76 columns and 454 nonzeros
Variable types: 60 continuous, 16 integer (16 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+01]
  Objective range  [1e+00, 4e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-01, 9e+01]

Loaded MIP start with objective 33
MIP start did not produce a new incumbent solution

Presolve removed 139 rows and 50 columns
Presolve time: 0.00s
Presolved: 58 rows, 26 columns, 170 nonzeros
Variable types: 20 continuous, 6 integer (6 binary)

Root relaxation: objective 0.000000e+00, 23 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    6   33.00000    0.00000   100%     -    0s
     0     0   13.17771    0    6   33.00000   13.17771  60.1%     -    0s
     0     0   17.00000    0    6   33.00000   17.00000  48.5%     -    0s
     0     0   22.00000    0    

### Get Solution

In [14]:
objective_value = getobjectivevalue(m)
arrivals = getvalue(a)
exits = getvalue(e)
ideals = t
;

### Print Solution

In [15]:
if status == :Optimal
    println("Objective value: ", objective_value)
    println("Arrival schedule: ", arrivals)
    println("Runway exit times: ", exits)
else
    println("Status: ", status)
end

Objective value: 33.0
Arrival schedule: [7.0, 5.0, 11.0, 3.0]
Runway exit times: [9.0, 5.5, 12.0, 5.0]


# Post-processing

### Compute derived values

In [21]:
event_times = sort(union(ideals,arrivals,exits))
T = length(event_times)

is_ideally_arrived = Array{Bool}(P, T)
is_arrived = Array{Bool}(P, T)
is_finished = Array{Bool}(P, T)
for p=1:P
    is_ideally_arrived[p,:] = (event_times .>= ideals[p])
    is_arrived[p,:] = (event_times .>= arrivals[p])
    is_finished[p,:] = (event_times .>= exits[p])
end
is_delayed = is_ideally_arrived .& .!is_arrived
is_landing = is_arrived .& .!is_finished
;

### Display Schedule

In [22]:
print("\n")
event_line = "|"
for ev = 1:T
    added = "--$(round(event_times[ev],1))"
    event_line *= added * "-"^(7-length(added))
end
event_line *= "--|\n"
print(event_line)

for p = 1:P
    plane_line = "|"
    plane_land = false
    for ev = 1:T
        if is_landing[p,ev]
            if plane_land
                plane_line *= "======="
            else
                plane_land = true
                plane_line *= "---<==="
            end
        else
            if plane_land
                plane_land = false
                plane_line *= "===>---"
            else
                plane_line *= "-------"
            end
        end
    end
    plane_line *= "--|\n"
    print(plane_line)
end


|--1.0----2.0----3.0----5.0----5.5----7.0----9.0----11.0---12.0---|
|--------------------------------------<======>-------------------|
|------------------------<======>---------------------------------|
|----------------------------------------------------<======>-----|
|-----------------<======>----------------------------------------|
